#Final Assignment
ntroduction where you discuss the business problem and who would be interested in this project.

Data where you describe the data that will be used to solve the problem and the source of the data.

Methodology section which represents the main component of the report where you discuss and describe any exploratory data analysis that you did, any inferential statistical testing that you performed, if any, and what machine learnings were used and why.

Results section where you discuss the results.

Discussion section where you discuss any observations you noted and any recommendations you can make based on the results.

Conclusion section where you conclude the report.

In [4]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import requests
import time
%pip install geocoder
import geocoder

     |████████████████████████████████| 102kB 2.2MB/s 


In [ ]:
df = pd.read_csv('population by town-city.csv')
df['Size'] = ""
df.head
 
for index, row in df.iterrows():
  df.iloc[index, 0] = df.iloc[index, 0].replace("buadisp2011:", "")
  df.iloc[index, 0] = df.iloc[index, 0].replace(" BUASD", "")
  df.iloc[index, 0] = df.iloc[index, 0].replace(" BUA", "")
  string = df.iloc[index, 0]
  i = 0
  i = string.find(" - ")
  if i != -1:
    df.iloc[index, 0] = string[i+3:]
 
df.head

<bound method NDFrame.head of                  Area   mnemonic    2011    2012  ...    2017    2018    2019  Size
0               Kexby  E34000001     342     342  ...     364     352     337      
1     Bradfield Heath  E34000002     724     715  ...     821     838     831      
2             Huntley  E34000003    1108    1121  ...    1127    1108    1131      
3          Swinefleet  E34000004     787     758  ...     787     794     770      
4            Foulsham  E34000005    1021     995  ...    1017     987     965      
...               ...        ...     ...     ...  ...     ...     ...     ...   ...
6616       Pontardawe  W38000146   12349   12357  ...   12801   12970   13096      
6617          Newport  W38000147  128115  128855  ...  133353  134920  136078      
6618       Bryn Pydew  W38000149     448     477  ...     454     448     449      
6619      Pentre Berw  W38000150     392     397  ...     404     421     416      
6620      Mynydd-bach  W38000153     391     3

In [ ]:
# create a list of our conditions
conditions = [(df['2019'] >= 7500) & (df['2019'] <= 24999),
    (df['2019'] >= 25000) & (df['2019'] <= 59999)
    ]

# create a list of the values we want to assign for each condition
values = ['Small Town', 'Medium Town']

# create a new column and use np.select to assign values to it using our lists as arguments
df['Size'] = np.select(conditions, values)


size_df = df[df['Size'] != "0"]
size_df.reset_index(inplace=True, drop=True)

In [ ]:
size_df['Latitude'] = ""
size_df['Longitude'] = ""
 
 
for index, row in size_df.iterrows():
  latlong = None
  while latlong == None:
    g = geocoder.arcgis('{}, United Kingdom'.format(row['Area']))
    latlong = g.latlng
  lat = str(latlong[0])
  long = str(latlong[1])
  size_df.iloc[index, 13] = long[:10]
  size_df.iloc[index, 12] = lat[:9]
 
size_df.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do

,Area,mnemonic,2011,2012,2013,2014,2015,2016,2017,2018,2019,Size,Latitude,Longitude
0,Dorchester (West Dorset),E34000016,19031,19152,19521,19633,19750,20254,20633,21082,21438,Small Town,50.710920,-2.4423399
1,Ely,E34000020,19270,19591,19543,19948,20053,20144,20255,20321,20330,Small Town,52.401370,0.26370000
2,Downham Market,E34000027,10882,10974,11192,11349,11650,11841,11937,11871,11910,Small Town,52.602360,0.37877000
3,Penrith,E34000039,15172,15232,15267,15287,15324,15376,15494,15542,15880,Small Town,54.664750,-2.7569299
4,Bolsover,E34000048,11767,12002,12042,12104,12052,12019,12014,11984,11996,Small Town,53.229250,-1.2907199


In [ ]:
CLIENT_ID = '31FBQ4JXECZV0OKLQFEDFTWIKMA0CCKDTNJ3JFCMTQLNQHT3' # your Foursquare ID
CLIENT_SECRET = 'UHA3UUT4XUGCX5ABTQSH41WXLGNQUISCE35RFSFCG1HHP5LG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100
radius = 5000
categoryId= '52f2ab2ebcbc57f1066b8b46'
 
# create URL
def create_url(lat, long):
  url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{},&radius={}&limit={}&categoryId={}'.format(
      CLIENT_ID, 
      CLIENT_SECRET, 
      VERSION, 
      lat,
      long,
      radius, 
      LIMIT,
      categoryId)
  return url
 
 
res_df = pd.DataFrame(columns=['Area', 'Venue Category', 'Venue Name', 'Longitude', 'Latitude'])
temp_df = res_df
temp_df.at[1, :] = ""
for index, row in size_df.iterrows():
  temp = temp_df
  lat = row['Latitude']
  long = row['Longitude']
  url = create_url(lat, long)
  try:
    results = requests.get(url).json()["response"]['groups'][0]['items']
    time.sleep(1)
    for res in results:
      temp['Area'] = row['Area']
      temp['Venue Name'] = res['venue']['name']
      temp['Venue Category'] = res['venue']['categories'][0]['name']
      temp['Longitude'] = res['venue']['location']['labeledLatLngs'][0]['lng']
      temp['Latitude'] = res['venue']['location']['labeledLatLngs'][0]['lat']
      res_df = res_df.append(temp, ignore_index=True)
  except:
    print("Skipping {}".format(index))

In [ ]:
res_df = res_df[res_df['Venue Category'] == 'Supermarket']
res_df.to_csv('results.csv')

In [ ]:
print(res_df.shape)
res_df.drop_duplicates(inplace=True)

(4780, 5)


In [61]:
res_df = pd.read_csv('results.csv')
print("Initial amount of unique names: {}".format(len(res_df['Venue Name'].unique().tolist())))

clean_df = res_df
search_name = ['orriso','o-op', 'press', 'aitrose', 'ermark', 'Tesco Superstore', 'Warehouse']
new_name = ['Morrisons', 'Co-op', 'Tesco Express', 'Waitrose & Partners', 'Generic Supermarket', 'Tesco Extra', 'Iceland']

for index, row in clean_df.iterrows():
  for num, name in enumerate(search_name):
    if name in row['Venue Name']:
      clean_df.loc[index, 'Venue Name'] = new_name[num]

print("Post Cleaning amount of unique names: {}".format(len(res_df['Venue Name'].unique().tolist())))

a = clean_df['Venue Name'].value_counts()
drop_list = []
for num, count in enumerate(a):
  if count < 10:
    drop_list.append(a.index[num])

#create drop series and specify types to keep
drop_index = []

#remove unwanted data
print("Removing unwanted Data")
for index, row in clean_df.iterrows():
  if row['Venue Name'] in drop_list:
    drop_index.append(index)

#clear unwanted data from 05 system type            
clean_df.drop(index = drop_index, axis =1, inplace=True)
clean_df.reset_index()


print("Post Cleaning low count names: {}".format(len(res_df['Venue Name'].unique().tolist())))

print(clean_df['Venue Name'].unique().tolist())


Initial amount of unique names: 106
Post Cleaning amount of unique names: 51
Removing unwanted Data
Post Cleaning low count names: 15
['Waitrose & Partners', 'Lidl', 'Tesco', "Sainsbury's", 'Aldi', 'Morrisons', 'Booths', 'Tesco Extra', 'Asda', "Sainsbury's Local", 'Iceland', 'Co-op', 'Tesco Express', 'Generic Supermarket', 'Nisa Local']


In [62]:
clean_df.to_csv('clean.csv')

In [ ]:
 num_top_venues = 10
 
for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

# New section